In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)

In [ ]:
import pandas as pd
data=pd.read_excel(r"/content/AirQualityUCI.xlsx")
print(data.isnull().sum())
data = data.fillna(method='ffill')
data = data.fillna(method='bfill')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12, 6))
data['NOx(GT)'].plot(title='AQI Over Time', ylabel='NOx(GT)')
from statsmodels.tsa.seasonal import seasonal_decompose
plt.show()

In [ ]:
data["year"]=data["Date"].dt.year
data["month"]=data["Date"].dt.month
data["day"]=data["Date"].dt.dayofweek
data["quarter"]=data["Date"].dt.quarter
data.head(5)

In [ ]:
data["NO2(GT)average"]=data["NO2(GT)"].rolling(window=24).mean()
data["NO2(GT)median"]=data["NO2(GT)"].rolling(window=24).median()
data["NO2(GT)sum"]=data["NO2(GT)"].rolling(window=24).sum()
data["NO2(GT)lag"]=data["NO2(GT)"].shift(24)#lag
data = data.dropna()
data.head(2)

In [ ]:
# for features
X = data.drop(columns=['NO2(GT)'])

if 'timestamp' in X.columns:
    X['hour'] = X['timestamp'].dt.hour
    X['day'] = X['timestamp'].dt.day
    X['month'] = X['timestamp'].dt.month
    X = X.drop(columns=['timestamp'])


X = X.select_dtypes(include=['number'])

# Split train test split
y = data['NO2(GT)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train_lstm = np.expand_dims(X_train.values, axis=-1).astype('float32')
X_test_lstm = np.expand_dims(X_test.values, axis=-1).astype('float32')

# Convert label col to float32
y_train = y_train.values.astype('float32')
y_test = y_test.values.astype('float32')
seq = Sequential([LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])), Dense(1)
])
seq.compile(optimizer='adam', loss='mse')
seq.fit(X_train_lstm, y_train, epochs=20, batch_size=32, validation_data=(X_test_lstm, y_test))

# ARIMA model for comparison
from statsmodels.tsa.arima.model import ARIMA
arima_model = ARIMA(y_train, order=(5, 1, 0)).fit()


In [ ]:
# Ensure y_test retains its original index
y = data['NO2(GT)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
# LSTM predictions
lstm_preds = model.predict(X_test_lstm).flatten()
arima_preds = arima_model.forecast(steps=len(y_test))
# Plot actual vs. predicted
plt.figure(figsize=(12, 8))
plt.plot(y_test.index, y_test, label='Actual AQI', color='blue')
plt.plot(y_test.index, lstm_preds, label='LSTM Predictions', color='orange')
plt.plot(y_test.index, arima_preds, label='ARIMA Predictions', color='green')
plt.legend()
plt.title("Actual vs Predicted AQI")
plt.xlabel("Time")
plt.ylabel("AQI")
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

ls_rmse = np.sqrt(mean_squared_error(y_test, lstm_preds))
ls_mae = mean_absolute_error(y_test, lstm_preds)
print(f"LSTM - RMSE: {ls_rmse}, MAE: {ls_mae}")

ar_rmse = np.sqrt(mean_squared_error(y_test, arima_preds))
ar_mae = mean_absolute_error(y_test, arima_preds)
print(f"ARIMA - RMSE: {ar_rmse}, MAE: {ar_rmse}")